In [65]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform, expon, reciprocal

In [66]:
# dirs
DATA_DIR = "./load.csv"
TEST_PLOT_DIR = "./test_plots/svm/"
TRAINING_HISTORY_DIR = "./training_history/svm.png"

In [67]:
if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)
if not os.path.exists("./model"):
    os.makedirs("./model")
if not os.path.exists("./training_history"):
    os.makedirs("./training_history")

In [68]:
data = pd.read_csv(DATA_DIR)
# Convert the 'date' column to datetime format
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y/%m/%d %H:%M')

# Sort the data by date
groups = data.groupby(pd.Grouper(key='Timestamp', freq='D'))

In [69]:
train_df = []
for date, data in groups:
    train_df.append(list(data["Load"]))
train_df = np.array(train_df)

In [70]:
scaler = MinMaxScaler()
scaler.fit(train_df)
train_df = scaler.transform(train_df)

In [71]:
display(train_df)

array([[0.81048387, 0.70979899, 0.76529338, ..., 0.53458382, 0.59077381,
        0.70172911],
       [0.32258065, 0.24623116, 0.66292135, ..., 0.25791325, 0.41666667,
        0.36023055],
       [0.42204301, 0.40326633, 0.31585518, ..., 0.2919109 , 0.375     ,
        0.41642651],
       ...,
       [0.22849462, 0.56658291, 0.21473159, ..., 0.50410317, 0.51041667,
        0.20461095],
       [0.22849462, 0.56658291, 0.21473159, ..., 0.50410317, 0.51041667,
        0.20461095],
       [0.22849462, 0.56658291, 0.21473159, ..., 0.50410317, 0.51041667,
        0.20461095]])

In [72]:
def create_dataset(data):
    X, y = [], []
    for i in range(9, len(data) - 1):
        last_two_days = data[i-2:i].flatten()
        last_week_same_days = data[i-7-2:i-7].flatten()

        input_data = np.concatenate([last_two_days, last_week_same_days])
        X.append(input_data)

        next_day = data[i+1]
        y.append(next_day)

    return np.array(X), np.array(y)

In [73]:
X, y = create_dataset(train_df)
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

X shape: (355, 384)
y shape: (355, 96)


In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [76]:
svr = SVR(kernel='rbf', C=25, epsilon=0.05)
multioutput_svr = MultiOutputRegressor(svr)
multioutput_svr.fit(X_train, y_train)

MultiOutputRegressor(estimator=SVR(C=25, epsilon=0.05))

In [75]:
# param_dist = {
#     'estimator__C': reciprocal(0.1, 100),
#     'estimator__epsilon': expon(scale=0.1)
# }

# svr0 = SVR(kernel='rbf')
# multioutput_svr0 = MultiOutputRegressor(svr0)
# random_search0 = RandomizedSearchCV(estimator=multioutput_svr0, param_distributions=param_dist, n_iter=500, cv=4, verbose=2, random_state=42, n_jobs=-1)
# random_search0.fit(X_train, X_train)

# print("\nrandom_search0 Best parameters:", random_search0.best_params_)
# print("random_search0 Best score:", random_search0.best_score_)


# random_search0 Best parameters: {'estimator__C': 25.61889217260502, 'estimator__epsilon': 0.0004642784066342885}
# random_search0 Best score: 0.8593395722336404

In [77]:
y_pred = multioutput_svr.predict(X_test)

loss = mean_squared_error(y_test, y_pred)

print("-" * 86)
print(f'Deep Learnong Testing Loss: {loss:.4f}')
print("-" * 86)

pred_data = scaler.inverse_transform(y_pred)
actual_data = scaler.inverse_transform(y_test)

for i in range(actual_data.shape[0]):
    plt.figure(figsize=(16, 6))
    X = np.arange(1, actual_data.shape[1]+1, 1)
    y_p = pred_data[i]
    y_a = actual_data[i]
    e = mean_squared_error(y_p, y_a)
    plt.title(f"Day {i+1} prediction result Loss:{e}")
    plt.plot(X, y_p, label='Predict')
    plt.plot(X, y_a, label='Actual')
    plt.ylim(0, 30)
    plt.xlabel('Time step')
    plt.ylabel('Usage (kWh)')
    plt.legend()
    plt.savefig(TEST_PLOT_DIR+f"Day{i+1}.png")
    plt.close()

--------------------------------------------------------------------------------------
Deep Learnong Testing Loss: 0.0169
--------------------------------------------------------------------------------------
